# DataGrid

In [ ]:
import pandas as pd, traitlets as T, ipywidgets as W
from wxyz.datagrid.widget_datagrid import DataGrid

In [ ]:
def make_grid(GridClass=DataGrid, df=None, container=W.VBox):
    dg = GridClass(layout=dict(height="60vh", flex="1"))
    rando = None
    if df is not None:
        dg.value = df
    else:
        with W.Output():
            @W.interact
            def rando(rows=(1, 1000), cols=(1, 1000)):
                dg.value = pd.np.random.rand(rows, cols)
    pg = container([*([rando.widget] if rando else []), dg], layout=dict(flex="1"))
    pg.dg = dg
    return pg

In [ ]:
if __name__ == "__main__":
    display(make_grid())